This notebook is a - kind of - continuation from: inside_bar_explore.ipynb

In [52]:
import pandas as pd
import plotly.graph_objects as go
import utils
import datetime as dt
from dateutil.parser import *

In [53]:
df_trades = pd.read_csv("./Data/Pairs/USD_JPY_H4_trades.csv")

In [54]:
pair = "USD_JPY"
granularity = "M5"

In [55]:
df_raw = pd.read_csv(utils.get_hist_data_filename(pair, granularity))
df_raw.shape

(223553, 16)

In [56]:
non_nums = ['ticker', 'time', 'volume']
num_cols = [x for x in df_raw.columns if x not in non_nums]
df_raw[num_cols] = df_raw[num_cols].apply(pd.to_numeric)

In [57]:
df_trades["time"] = [parse(x) for x in df_trades.time]
df_raw["time"] = [parse(x) for x in df_raw.time]

We need to know the time of the next trade (shift), and the time of the last 5 minute trade in the 4 hour window (because the "H4" granularity data is what we're digging into).  So, that's why we're adding 03:55 to the start time of the 4 hour window.

In [58]:
df_trades["next"] =  df_trades["time"].shift(-1)
df_trades["trade_end"] = df_trades.next + dt.timedelta(hours=3, minutes=55)
df_trades[["time", "next", "trade_end"]].head()

,time,next,trade_end
0,2020-01-05 22:00:00+00:00,2020-01-06 22:00:00+00:00,2020-01-07 01:55:00+00:00
1,2020-01-06 22:00:00+00:00,2020-01-07 06:00:00+00:00,2020-01-07 09:55:00+00:00
2,2020-01-07 06:00:00+00:00,2020-01-09 18:00:00+00:00,2020-01-09 21:55:00+00:00
3,2020-01-09 18:00:00+00:00,2020-01-16 10:00:00+00:00,2020-01-16 13:55:00+00:00
4,2020-01-16 10:00:00+00:00,2020-01-20 14:00:00+00:00,2020-01-20 17:55:00+00:00


In [59]:
def signal_text(signal):
    if signal == 1:
        return "BUY"
    elif signal == -1:
        return "SELL"
    else:
        return "NONE"
    


#### triggered function:  Receive Buy/Sell direction, get current price from a finer granularity against higher granularity entry(signal) price.

Simplified example:
* We have a Entry Price of 1.00, identified in a 4 hour candle
* We meet 12 5 minute candles at < 1.00
* On candle 13 (an hour in) we get a price of 2.99
* We buy at 2.99, but our take profit is 3.00 - so this is a worthless trade.
* Alternately, candle 13 had a price of 1.01.  If we hit our take profit, we done well. 

In [60]:
def triggered(direction, current_price, signal_price):
    if direction == 1 and current_price > signal_price:
        return True
    elif direction == -1 and current_price < signal_price:
        return True
    else:
        return False

We've used iterrows to loop through DataFrames until now, but this can be slow.  The process_m5 function presents a faster alternative.

In [61]:
def process_m5(m5_df, row):
    for price in m5_df.mid_c.values:
        if triggered(row.SIGNAL, price, row.ENTRY) == True:
            print(f"   {signal_text(row.SIGNAL)} Signal at {price: 2f} from Entry: {row.ENTRY: 2f} ")           
            break


In [62]:
df_trades.dropna(inplace=True)
df_trades.reset_index(drop=True, inplace=True)

In [63]:
for index, row in df_trades.iterrows():
    m5_data = df_raw[(df_raw.time >= row.time) & (df_raw.time <= row.trade_end)]
    print(f"{index} {row.time} {signal_text(row.SIGNAL)} {row.ENTRY: 2f} {m5_data.shape}")
    process_m5(m5_data, row)
    if index > 10:
        break

0 2020-01-05 22:00:00+00:00 BUY  108.154400 (334, 16)
   BUY Signal at  108.157000 from Entry:  108.154400 
1 2020-01-06 22:00:00+00:00 BUY  108.522400 (142, 16)
2 2020-01-07 06:00:00+00:00 BUY  108.514300 (768, 16)
   BUY Signal at  108.537000 from Entry:  108.514300 
3 2020-01-09 18:00:00+00:00 BUY  109.595100 (1380, 16)
   BUY Signal at  109.596000 from Entry:  109.595100 
4 2020-01-16 10:00:00+00:00 BUY  110.079400 (671, 16)
   BUY Signal at  110.137000 from Entry:  110.079400 
5 2020-01-20 14:00:00+00:00 BUY  110.204200 (2542, 16)
   BUY Signal at  110.214000 from Entry:  110.204200 
6 2020-01-31 06:00:00+00:00 BUY  109.137100 (528, 16)
7 2020-02-03 22:00:00+00:00 BUY  108.760500 (335, 16)
   BUY Signal at  108.766000 from Entry:  108.760500 
8 2020-02-04 22:00:00+00:00 BUY  109.557600 (431, 16)
   BUY Signal at  109.678000 from Entry:  109.557600 
9 2020-02-06 06:00:00+00:00 BUY  109.996900 (718, 16)
   BUY Signal at  110.020000 from Entry:  109.996900 
10 2020-02-10 14:00:00+00:

In [64]:
m5_data.head()

,Unnamed: 0,ticker,time,volume,bid_o,bid_h,bid_l,bid_c,mid_o,mid_h,mid_l,mid_c,ask_o,ask_h,ask_l,ask_c
8283,24,USD_JPY,2020-02-11 18:00:00+00:00,148,109.808,109.826,109.798,109.824,109.814,109.832,109.804,109.830,109.820,109.839,109.810,109.837
8284,25,USD_JPY,2020-02-11 18:05:00+00:00,144,109.823,109.832,109.818,109.819,109.830,109.838,109.824,109.825,109.836,109.845,109.830,109.831
8285,26,USD_JPY,2020-02-11 18:10:00+00:00,85,109.817,109.828,109.816,109.825,109.824,109.834,109.822,109.831,109.830,109.839,109.828,109.837
8286,27,USD_JPY,2020-02-11 18:15:00+00:00,138,109.826,109.828,109.802,109.802,109.833,109.834,109.808,109.808,109.840,109.840,109.814,109.814
8287,28,USD_JPY,2020-02-11 18:20:00+00:00,120,109.803,109.807,109.794,109.798,109.809,109.812,109.800,109.804,109.815,109.818,109.806,109.809
